<a href="https://colab.research.google.com/github/wickedWOLF123/DRP/blob/main/GSM8k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tiktoken
!pip install transformers datasets sentencepiece accelerate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from datasets import load_dataset
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments,default_data_collator, GPT2Config
from torch.utils.data import Dataset, DataLoader, random_split
import tiktoken
import math


The objective of this project is to combine to test out a Trandformer model as a baseline for of Large Language Models before using  Reinforcement Learning techniques to solve math problems using Multistep reasoning techniques, increasing the test time compute. We look at the Graduate School Math dataset (GSM8k), originally posted as part of "Training Verifiers to Solve Math Word Problems". I also use the GSM-Plus on HuggingFace at https://huggingface.co/datasets/qintongli/GSM-Plus

In [ ]:
'''
  There are some special places within questions and answers where its better to replace
  with special tokens.
'''
SPECIAL_TOKENS = {
    "STEP_START": "[STEP_START]",
    "STEP_END": "[STEP_END]",
    "MATH_OP": "[MATH_OP]",
    "REASONING": "[REASONING]",
    "PROBLEM_START": "[PROBLEM_START]",
    "PROBLEM_END": "[PROBLEM_END]",
    "SOLUTION_START": "[SOLUTION_START]",
    "SOLUTION_END": "[SOLUTION_END]",
    "FINAL_ANSWER": "[FINAL_ANSWER]"
}

SEED = 42

random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)


Loading the dataset for HF


In [ ]:
NUM_SAMPLES = 3000

dataset = load_dataset("openai/gsm8k", "main")
dataset_2 = load_dataset("qintongli/GSM-Plus")

train_data = dataset['train']
train_data = train_data.shuffle(seed=SEED).select(range(min(NUM_SAMPLES, len(dataset['train']))))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

...

README.md:   0%|          | 0.00/3.41k [00:00<?, ?B/s]

test-00000-of-00001.jsonl:   0%|          | 0.00/13.3M [00:00<?, ?B/s]

data/testmini-00000-of-00001.jsonl:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/10552 [00:00<?, ? examples/s]

Generating testmini split:   0%|          | 0/2400 [00:00<?, ? examples/s]

In [ ]:
print(train_data[0])

{'question': 'Mimi picked up 2 dozen seashells on the beach.  Kyle found twice as many shells as Mimi and put them in his pocket. Leigh grabbed one-third of the shells that Kyle found.  How many seashells did Leigh have?', 'answer': 'Mimi has 2 x 12 = <<2*12=24>>24 sea shells.\nKyle has 24 x 2 = <<24*2=48>>48 sea shells.\nLeigh has 48 / 3 = <<48/3=16>>16 sea shells.\n#### 16'}


Setting up encoding using Tiktoken with o2000_base which is encodign from GPT-4o

In [ ]:
enc = tiktoken.get_encoding("o200k_base")

vocab_size  = enc.n_vocab

def format_example(example):
    question = example['question'].strip()
    answer = example['answer'].strip()
    text = (
        f"{SPECIAL_TOKENS['PROBLEM_START']} {question} {SPECIAL_TOKENS['PROBLEM_END']} "
        f"{SPECIAL_TOKENS['SOLUTION_START']} "
        f"{SPECIAL_TOKENS['STEP_START']} {SPECIAL_TOKENS['REASONING']} {answer} {SPECIAL_TOKENS['STEP_END']} "
        f"{SPECIAL_TOKENS['FINAL_ANSWER']} {answer} {SPECIAL_TOKENS['SOLUTION_END']}"
    )
    return text

def encode_text(text):
    return enc.encode_ordinary(text)

print(format_example(train_data[0]))
print(encode_text(format_example(train_data[0])))
print(len(encode_text(format_example(train_data[0]))))

[PROBLEM_START] Mimi picked up 2 dozen seashells on the beach.  Kyle found twice as many shells as Mimi and put them in his pocket. Leigh grabbed one-third of the shells that Kyle found.  How many seashells did Leigh have? [PROBLEM_END] [SOLUTION_START] [STEP_START] [REASONING] Mimi has 2 x 12 = <<2*12=24>>24 sea shells.
Kyle has 24 x 2 = <<24*2=48>>48 sea shells.
Leigh has 48 / 3 = <<48/3=16>>16 sea shells.
#### 16 [STEP_END] [FINAL_ANSWER] Mimi has 2 x 12 = <<2*12=24>>24 sea shells.
Kyle has 24 x 2 = <<24*2=48>>48 sea shells.
Leigh has 48 / 3 = <<48/3=16>>16 sea shells.
#### 16 [SOLUTION_END]
[58, 15513, 134327, 28418, 60, 152459, 19148, 869, 220, 17, 38449, 67005, 10844, 82, 402, 290, 11334, 13, 220, 61955, 2491, 18370, 472, 1991, 84266, 472, 152459, 326, 3006, 1373, 306, 1232, 25546, 13, 117479, 56238, 1001, 54370, 328, 290, 84266, 484, 61955, 2491, 13, 220, 3253, 1991, 67005, 10844, 82, 2242, 117479, 679, 30, 723, 15513, 134327, 23987, 60, 723, 50, 102002, 28418, 60, 723, 91410, 2

In [ ]:
# Peform encoding on chose training data along with formating
# Max len after performing is capped at 512 for colab
MAX_LENGTH = 256

encoded_samples = []
for ex in train_data:
    text = format_example(ex)
    ids = encode_text(text)
    ids = ids[:MAX_LENGTH]
    encoded_samples.append(ids)


In [ ]:
print(encoded_samples[0])

[58, 15513, 134327, 28418, 60, 152459, 19148, 869, 220, 17, 38449, 67005, 10844, 82, 402, 290, 11334, 13, 220, 61955, 2491, 18370, 472, 1991, 84266, 472, 152459, 326, 3006, 1373, 306, 1232, 25546, 13, 117479, 56238, 1001, 54370, 328, 290, 84266, 484, 61955, 2491, 13, 220, 3253, 1991, 67005, 10844, 82, 2242, 117479, 679, 30, 723, 15513, 134327, 23987, 60, 723, 50, 102002, 28418, 60, 723, 91410, 28418, 60, 723, 1099, 69256, 2694, 60, 152459, 853, 220, 17, 1215, 220, 899, 314, 2256, 17, 9, 899, 28, 1494, 3920, 1494, 9624, 84266, 558, 192951, 853, 220, 1494, 1215, 220, 17, 314, 2256, 1494, 9, 17, 28, 3519, 3920, 3519, 9624, 84266, 558, 2486, 1251, 853, 220, 3519, 820, 220, 18, 314, 2256, 3519, 14, 18, 28, 1125, 3920, 1125, 9624, 84266, 558, 1509, 220, 1125, 723, 91410, 23987, 60, 723, 176019, 62, 160761, 60, 152459, 853, 220, 17, 1215, 220, 899, 314, 2256, 17, 9, 899, 28, 1494, 3920, 1494, 9624, 84266, 558, 192951, 853, 220, 1494, 1215, 220, 17, 314, 2256, 1494, 9, 17, 28, 3519, 3920, 3519

In [ ]:
# Creating a Pytorch dataset for ease of model

class TokenizedDataset(Dataset):
    def __init__(self, token_ids_list, max_length=MAX_LENGTH, pad_token_id=0):
        self.token_ids_list = token_ids_list
        self.max_length = max_length
        self.pad_token_id = pad_token_id

    def __len__(self):
        return len(self.token_ids_list)

    def __getitem__(self, idx):
        input_ids = self.token_ids_list[idx]
        # Pad sequences to max_length
        pad_len = self.max_length - len(input_ids)
        input_ids = input_ids + [self.pad_token_id] * pad_len
        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "labels": torch.tensor(input_ids, dtype=torch.long)
        }

# Determine pad_token_id
pad_token_id = enc.eot_token if hasattr(enc, 'eot_token') else 0
train_dataset = TokenizedDataset(encoded_samples, max_length=MAX_LENGTH, pad_token_id=pad_token_id)


In [ ]:
print(train_dataset[0])

{'input_ids': tensor([    58,  15513, 134327,  28418,     60,   9312,   1058,  38147,  22639,
           548,     15,     13,   1434,    501,   1481,    810,    316,    290,
         14194,   2910,    326,   3877,    261,  42573,   3608,     13,    220,
         38147,    885,  13034,   2059,    501,   1481,   2777,   2395,    548,
            15,     13,   3384,   1753,   2384,    395,   6167,    842,   1232,
         26508,     13,    220,   3604,    290,   1268,    328,   4242,   7449,
            11,   1495,   1991,  42573,  21821,    738,  38147,    413,   3741,
           316,   3877,     30,    723,  15513, 134327,  23987,     60,    723,
            50, 102002,  28418,     60,    723,  91410,  28418,     60,    723,
          1099,  69256,   2694,     60,   1679,    382,   9927,    548,     15,
            13,   3384,   1753,   2384,    395,    220,     19,   7449,     11,
           813,  38147,   2452,    887,   3384,    425,    220,     19,    314,
           548,   5354,   

In [ ]:
eval_size = 100  # Number of samples for evaluation
train_size = len(train_dataset) - eval_size

train_dataset, eval_dataset = random_split(
    train_dataset,
    [train_size, eval_size],
    generator=torch.Generator().manual_seed(SEED)
)


print(f"Training dataset size: {len(train_dataset)}")
print(f"Evaluation dataset size: {len(eval_dataset)}")


Training dataset size: 2900
Evaluation dataset size: 100


In [ ]:
# Define the TransformerModel
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_length=512):
        super(PositionalEncoding, self).__init__()
        position = torch.arange(0, max_length, dtype=torch.float).unsqueeze(1)  # (max_length, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))  # (d_model/2,)
        pe = torch.zeros(max_length, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)  # Even indices
        pe[:, 1::2] = torch.cos(position * div_term)  # Odd indices
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        # x: (batch_size, seq_len, d_model)
        seq_len = x.size(1)
        x = x + self.pe[:, :seq_len, :].to(x.device)
        return x

class TransformerModel(nn.Module):
    def __init__(self, vocab_size, max_length, d_model, nhead, num_layers):
        super(TransformerModel, self).__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_length)

        # Define Transformer Encoder with masked self-attention for decoder-only architecture
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=d_model * 4, dropout=0.1)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.generator = nn.Linear(d_model, vocab_size)

        # Generate square subsequent mask for causal masking
        self.register_buffer("mask", torch.tril(torch.ones(MAX_LENGTH, MAX_LENGTH)))

    def forward(self, input_ids, labels=None):
        embedded = self.embedding(input_ids) * math.sqrt(self.d_model)  # (batch_size, seq_len, d_model)
        embedded = self.positional_encoding(embedded)  # (batch_size, seq_len, d_model)
        embedded = embedded.transpose(0, 1)  # (seq_len, batch_size, d_model)

        # Create causal mask
        seq_len = embedded.size(0)
        causal_mask = self.mask[:seq_len, :seq_len].to(embedded.device)  # (seq_len, seq_len)
        causal_mask = causal_mask.masked_fill(causal_mask == 0, float('-inf')).masked_fill(causal_mask == 1, float(0.0))  # Replace 0 with -inf and 1 with 0.0

        # Transformer Encoder expects src and mask
        output = self.transformer_encoder(embedded, mask=causal_mask)

        output = output.transpose(0,1)  # (batch_size, seq_len, d_model)
        logits = self.generator(output)  # (batch_size, seq_len, vocab_size)

        loss = None
        if labels is not None:
            # Shift logits and labels for next-token prediction
            loss_fct = nn.CrossEntropyLoss(ignore_index=pad_token_id)
            # Reshape logits and labels to (batch_size * seq_len, vocab_size) and (batch_size * seq_len)
            loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))

        return (loss, logits) if loss is not None else logits


In [ ]:
# Initialize the Transformer Model
D_MODEL = 256
N_HEAD = 8
NUM_LAYERS = 6
BATCH_SIZE = 1

# Initialize the Transformer Model
model = TransformerModel(
    vocab_size=vocab_size,
    max_length=MAX_LENGTH,
    d_model=D_MODEL,
    nhead=N_HEAD,
    num_layers=NUM_LAYERS
)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print(f"Model initialized on {device}.")

# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./math_transformer_model",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    eval_strategy="epoch",
    save_steps=500,
    logging_steps=100,
    learning_rate=5e-4,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./logs_math_transformer",
    report_to="none",  # Disable reporting to external services
    save_total_limit=2,
)

# Define a Wrapper to Match Trainer's Expected Output
class ModelWrapper(nn.Module):
    def __init__(self, model):
        super(ModelWrapper, self).__init__()
        self.model = model

    def forward(self, input_ids, labels=None):
        outputs = self.model(input_ids, labels=labels)
        if labels is not None:
            return {"loss": outputs[0], "logits": outputs[1]}
        else:
            return {"logits": outputs}

# Wrap the Model
wrapped_model = ModelWrapper(model)

# Initialize Trainer
trainer = Trainer(
    model=wrapped_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
)

# Start Training
trainer.train()


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Model initialized on cuda.


Epoch,Training Loss,Validation Loss
1,0.434000,0.377028
2,0.168600,0.256980
3,0.049500,0.239419


TrainOutput(global_step=8700, training_loss=0.47158182155126815, metrics={'train_runtime': 850.6104, 'train_samples_per_second': 10.228, 'train_steps_per_second': 10.228, 'total_flos': 0.0, 'train_loss': 0.47158182155126815, 'epoch': 3.0})

In [ ]:
# Switch Model to Evaluation Mode
model.eval()

# Define a Prompt
prompt = (
    f"{SPECIAL_TOKENS['PROBLEM_START']} Josh decides to try flipping a house. He buys a house for $80,000 and then puts in $50,000 in repairs. This increased the value of the house by 150%. How much profit did he make? {SPECIAL_TOKENS['PROBLEM_END']} "
    f"{SPECIAL_TOKENS['SOLUTION_START']} {SPECIAL_TOKENS['STEP_START']} {SPECIAL_TOKENS['REASONING']}"
)

# Encode the Prompt
input_ids = enc.encode_ordinary(prompt)
input_ids = input_ids[:MAX_LENGTH]  # Ensure max length
input_ids = input_ids + [enc.eot_token] * (MAX_LENGTH - len(input_ids)) if len(input_ids) < MAX_LENGTH else input_ids[:MAX_LENGTH]

# Convert to Tensor and Add Batch Dimension
input_ids_tensor = torch.tensor([input_ids], dtype=torch.long).to(device)

# Generate Output
with torch.no_grad():
    logits = model(input_ids_tensor)
    # Apply softmax to get probabilities
    probabilities = F.softmax(logits, dim=-1)
    # Get the most probable next token
    predicted_token_id = torch.argmax(probabilities, dim=-1)
    # Decode the token
    generated_text = enc.decode(predicted_token_id[0].cpu().numpy())

print("Generated solution:\n", generated_text)


Generated solution:
 [PROBLEM_START] Josh decides to try2 a house. He buys a house for $80,000 and then puts in $50,000 in repairs. This increased the value of the house by 150%. How much profit did he make? [PROBLEM_END] [SOLUTION_START] [STEP_START] [REASONING] spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces spaces Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Trey Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Char Ch